In [ ]:
from sklearn.cluster import DBSCAN
from shapely.geometry import MultiPoint,Point,Polygon
import pandas as pd
import geopandas as gp
from shapely import wkt
import numpy as np
import matplotlib.pyplot as plt
from watertap3.truck_pipe_cost_functions import pipe_costing

from sklearn.cluster import DBSCAN
from shapely import wkt

1. Get location of brackish water wells
2. Assume an epsilon range and find the centroid
3. Calculate LCOW for cost of treatment +  transport cost (from well to centroid)
4. Plot LCOW as a function of epsilon

In [ ]:
bw_df = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/brackish_sites_with_metrics_baseline_dwi_updated_costs_transport_updated_basis.csv')
texas = gp.read_file('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/GeoData/Texas Counties Map/geo_export_c05b3355-1638-4e46-b56c-ac18de751ed8.shp')
texas = texas.to_crs("EPSG:4326")

In [ ]:
bw_long = bw_df[bw_df['state_alpha']=='TX']['Longitude']
bw_lat = bw_df[bw_df['state_alpha']=='TX']['Latitude']

bw_geometry = [Point(xy) for xy in zip(bw_long,bw_lat)]
bw_geo_df = gp.GeoDataFrame(geometry = bw_geometry, crs='EPSG:4326')
bw_geo_df.geometry = bw_geo_df.geometry.to_crs('EPSG:4326')
bw_geo_df = bw_geo_df.assign(county_nm = bw_df[bw_df['state_alpha'] =='TX']['county_nm'].values)

In [ ]:
coords = bw_df[bw_df['state_alpha']=='TX'][['Latitude', 'Longitude']].to_numpy()

In [ ]:
# define the number of kilometers in one radian
kms_per_radian = 6371.0088
epsilon = 25/ kms_per_radian

db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_

# get the number of clusters
num_clusters = len(set(cluster_labels))

# turn the clusters in to a pandas series, where each element is a cluster of points
clusters = pd.Series((coords[cluster_labels==n] for n in range(num_clusters)))
centroids = []

for ea in clusters:
    centroids.append((MultiPoint(ea).centroid.x, MultiPoint(ea).centroid.y))

centroids = np.array(centroids)

print(num_clusters)

In [ ]:
loc = [Point(xy) for xy in zip(centroids[:,1],centroids[:,0])]
loc = gp.GeoDataFrame(geometry = loc, crs='EPSG:4326')
loc.geometry = loc.geometry.to_crs('EPSG:4326')

Add cluster labels to brackish water sites

In [ ]:
bw_df_tx = bw_df[bw_df['state_alpha']=='TX']
bw_df_tx['cluster_label'] = cluster_labels

for x in range(len(bw_df_tx)):
    centroid_list = loc.geometry[bw_df_tx.iloc[x]['cluster_label']]

bw_df_tx['centroid'] = centroid_list

bw_df_tx.head()

In [ ]:
# Checking for duplications
# dup = bw_df_tx[bw_df_tx.duplicated(['unique_site_ID'],keep = 'first' )]
# dup

In [ ]:
fig, (ax,ax0) = plt.subplots(1,2,figsize = (20,20))
# states[states['NAME'] == 'Texas'].plot(ax=ax,figsize=(20, 20))
texas.plot(ax=ax)
bw_geo_df.plot(ax=ax, color='purple', markersize = 5,label = "BW Sites")

texas.plot(ax=ax0)
bw_geo_df.plot(ax=ax0, markersize = 10,edgecolors = 'black', c='purple',linewidth = 0.1, label = "BW Sites")
ax0.scatter(centroids[:,1],centroids[:,0],color='black',s = 2)

In [ ]:
bw_df.head()

Brackish Water Supply Curve

In [ ]:
bw_df_tx[bw_df_tx["cluster_label"]==0]

Trying to create treatment train files

In [ ]:
cols = ['CaseStudy','Reference','Scenario','Unit','Type','UnitName','ToUnitName','FromPort','Parameter']

train_df = pd.DataFrame(columns= cols)

for